<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/4%20-%20Preprocessing%20del%20testo/codifica_testo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Codifica del testo
In questo notebook vedremo come implementare le due tecniche principali per la codifica del testo in numeri, **Bag of Words** e **TF-IDF**.

## Bag of Words
Il modello Bag of Words consiste nel creare un vocabolario di tutte le parole presenti all'interno del nostro corpus di testo, per poi contare quante volte ognuna di esse compare all'interno di un documento. Dato che un singolo documento contiene solo una minuscola parte dell'intero vocabolario, il risultato del bag of words sarà una matrice sparsa, cioè una matrice composta per lo più da zeri.
<br>
Definiamo un piccolo corpus di testo che useremo per un'esempio.

In [0]:
corpus = ["la mamma prepara la pasta", "la nonna prepara la pizza", "papà guarda la mamma"]

Definiamo una funzione che estrae i tokens da ogni documento all'interno del corpus.

In [0]:
def corpus_tokenizer(corpus):
  tokens = [sent.split() for sent in corpus] 
  return tokens

Definiamo una funzione che creerà il dizionario

In [0]:
def build_vocab(corpus_tokens):
  
  # usando un set evitiamo
  # l'inserimento di duplicati
  vocab = set({})
  
  for tokens in corpus_tokens:
    for token in tokens:
      vocab.add(token)
    
  return list(vocab)

Adesso definiamo la funzione per eseguire il bag of words, la matrice risultate avrà un numero di righe pari al numero di documenti e un numero di colonne pari al numero di parole all'interno del vocabolario.

In [0]:
def bag_of_words(corpus):

  corpus_tokens = corpus_tokenizer(corpus)
  vocab = build_vocab(corpus_tokens)
  
  corpus_bow = []

  for tokens in corpus_tokens:
    sent_bow = []
  
    for word in vocab:
      sent_bow.append(tokens.count(word))
    corpus_bow.append(sent_bow)
  
  return corpus_bow, vocab

In [24]:
corpus_bow, vocab = bag_of_words(corpus)

print("Dizionario:",vocab)
print("\n")

for sent, bow in zip(corpus, corpus_bow):
  print("Frase:", sent)
  print("Bag of Words:", bow)
  print("-------")

Dizionario: ['guarda', 'nonna', 'prepara', 'pizza', 'pasta', 'papà', 'mamma', 'la']


Frase: la mamma prepara la pasta
Bag of Words: [0, 0, 1, 0, 1, 0, 1, 2]
-------
Frase: la nonna prepara la pizza
Bag of Words: [0, 1, 1, 1, 0, 0, 0, 2]
-------
Frase: papà guarda la mamma
Bag of Words: [1, 0, 0, 0, 0, 1, 1, 1]
-------


## Bag of Words con Numpy
Vediamo ora un'implementazione del bag of words che sfrutta numpy, importiamo la libreria, in questo caso abbiamo creato un vocabolario inverso che mappa una parola al corrispondente indice all'interno del vocabolario tramite un dizionario, così facendo possiamo eseguire la codifica iterando sui token piuttosto che su tutte le parole del vocabolario. Questa tecnica porta un risparmio di tempo notevole quando il vocabolario è di grandi dimensioni.

In [0]:
import numpy

def bag_of_words_np(corpus):

  corpus_tokens = corpus_tokenizer(corpus)
  
  index_to_word = build_vocab(corpus_tokens)
  word_to_index = dict([(char, i) for i, char in enumerate(index_to_word)]) # creiamo il vocabolario inverso
  
  docs_count = len(corpus)
  vocab_size = len(index_to_word)
  
  corpus_bow = np.zeros((docs_count, vocab_size))

  for i, tokens in enumerate(corpus_tokens):
    for token in tokens:
      corpus_bow[i][word_to_index[token]]+=1 # usiamo il vocabolario inverso
  
  return corpus_bow, vocab

Testiamo la funzione, come vedi il risultato è lo stesso di prima.

In [64]:
corpus_bow, vocab = bag_of_words_np(corpus)

print("Dizionario:",vocab)
print("\n")

for sent, bow in zip(corpus, corpus_bow):
  print("Frase:", sent)
  print("Bag of Words:", bow)
  print("-------")

Dizionario: ['guarda', 'nonna', 'prepara', 'pizza', 'pasta', 'papà', 'mamma', 'la']


Frase: la mamma prepara la pasta
Bag of Words: [0. 0. 1. 0. 1. 0. 1. 2.]
-------
Frase: la nonna prepara la pizza
Bag of Words: [0. 1. 1. 1. 0. 0. 0. 2.]
-------
Frase: papà guarda la mamma
Bag of Words: [1. 0. 0. 0. 0. 1. 1. 1.]
-------


## TF-IDF
Il TF-IDF, abbreviazione di Term Frequency - Inverse Document Frequency è un modello simile al bag of words, ma che tende a penalizzare le parole comuni all'interno del corpus di testo e a dare maggiore importanza a quelle più rare.
<br>
Il TF-IDF è composto dai seguenti elementi:
- **Term Frequency**: quante volte un termine appare all'interno di un documento.
- **Document Frequency**: in quanti documenti appare un termine.
- **Inverse Document Frequency**: è il logaritmo del rapporto tra il numero di documenti e il document frequency, più uno se vogliamo evitare che le parole presenti in ogni documento abbiamo un IDF di 0.

Il TF-IDF è dato dal prodotto del Term Frequency e l'Inverse Document Frequency.

In [0]:
corpus = ["la mamma prepara la pasta", "la nonna prepara la pizza", "papà guarda la mamma"]

In [0]:
from math import log

def tf_idf(corpus):
  
  corpus_tokens = corpus_tokenizer(corpus)
  vocab = build_vocab(corpus_tokens)

  # Document Frequency
    
  df = [0]*len(vocab)

  for i,word in enumerate(vocab):
    for tokens in corpus_tokens:
      if(word in tokens):
        df[i]+=1

  # Inverse Document frequency
        
  docs_count = len(corpus)
  idf = [log(docs_count/i)+1 for i in df]

  # Term Frequency
  
  tf = []

  for tokens in corpus_tokens:
    tf_sent = []
    words_count = len(tokens)
  
    for word in vocab:
      tf_sent.append(tokens.count(word)/words_count)
    tf.append(tf_sent)

  #TF-IDF
    
  tf_idf = []

  for i in range(docs_count):
    tf_idf.append([tf_i*idf_i for tf_i,idf_i in zip(tf[i],idf)])

  return tf_idf, vocab

Testiamo la nostra implementazione del TF-IDF.

In [85]:
corpus_tfidf, vocab = tf_idf(corpus)

print("Dizionario:",vocab)
print("\n")

for sent, tfidf in zip(corpus, corpus_tfidf):
  print("Frase:", sent)
  print("TF-IDF:", tfidf)
  print("-------")

Dizionario: ['guarda', 'nonna', 'prepara', 'pizza', 'pasta', 'papà', 'mamma', 'la']


Frase: la mamma prepara la pasta
TF-IDF: [0.0, 0.0, 0.2810930216216329, 0.0, 0.41972245773362205, 0.0, 0.2810930216216329, 0.4]
-------
Frase: la nonna prepara la pizza
TF-IDF: [0.0, 0.41972245773362205, 0.2810930216216329, 0.41972245773362205, 0.0, 0.0, 0.0, 0.4]
-------
Frase: papà guarda la mamma
TF-IDF: [0.5246530721670275, 0.0, 0.0, 0.0, 0.0, 0.5246530721670275, 0.3513662770270411, 0.25]
-------


##TF-IDF con Numpy
Implementiamo il TF-IDF nuovamente con Numpy, utilizziamo anche in questo caso la tecnica del vocabolario inverso.

In [0]:
def np_tf_idf(corpus):
  
  corpus_tokens = corpus_tokenizer(corpus)
  
  index_to_word = build_vocab(corpus_tokens)
  word_to_index = dict([(char, i) for i, char in enumerate(index_to_word)]) # creiamo il vocabolario inverso
  
  vocab_size = len(vocab)
  docs_count = len(corpus)
  
  # Document Frequency
    
  df = np.zeros(vocab_size)

  for i,word in enumerate(vocab):
    for tokens in corpus_tokens:
      if(word in tokens):
        df[i]+=1

  # Inverse Document frequency
        
  idf = np.log(docs_count/df)+1

  # Term Frequency
  
  tf = np.zeros((docs_count, vocab_size))
  
  for i, tokens in enumerate(corpus_tokens):
    word_counts = len(tokens)
    for token in tokens:
      tf[i][word_to_index[token]]+=1 # usiamo il vocabolario inverso
    tf[i]/=word_counts

  #TF-IDF
    
  tf_idf = tf*idf

  return tf_idf, vocab

Testiamo la nostra nuova implementazione.

In [97]:
corpus_tfidf, vocab = np_tf_idf(corpus)

print("Dizionario:",vocab)
print("\n")

for sent, tfidf in zip(corpus, corpus_tfidf):
  print("Frase:", sent)
  print("TF-IDF:", tfidf)
  print("-------")

Dizionario: ['guarda', 'nonna', 'prepara', 'pizza', 'pasta', 'papà', 'mamma', 'la']


Frase: la mamma prepara la pasta
TF-IDF: [0.         0.         0.28109302 0.         0.41972246 0.
 0.28109302 0.4       ]
-------
Frase: la nonna prepara la pizza
TF-IDF: [0.         0.41972246 0.28109302 0.41972246 0.         0.
 0.         0.4       ]
-------
Frase: papà guarda la mamma
TF-IDF: [0.52465307 0.         0.         0.         0.         0.52465307
 0.35136628 0.25      ]
-------


Il risultato è lo stesso di prima, ma questa implementazione performa meglio nel caso di corpus di testo molto grandi.